<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/TTPA_EXA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai -q
!pip install exa-py -q
!pip install google-colab -q
!pip install openai -q
!pip install crewai[tools] -q

## weatherAPI

In [32]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from exa_py import Exa
from google.colab import userdata
from langchain_openai import ChatOpenAI
from datetime import datetime, timedelta

# --- 0. Set Up Your Environment Variables (IMPORTANT!) ---
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["EXA_API_KEY"] = userdata.get('EXA_API_KEY')

# --- 1. Define Custom Exa Tools ---

class ExaSearchTool(BaseTool):
    name: str = "Exa Search Tool"
    description: str = "Searches the web using Exa for relevant information, returning titles, URLs, and IDs."
    exa_instance: Exa = None

    def __init__(self):
        super().__init__()
        try:
            self.exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
        except KeyError:
            raise ValueError("EXA_API_KEY environment variable not set. Please set it to your Exa API key in Colab Secrets.")
        except Exception as e:
            raise Exception(f"Failed to initialize ExaSearchTool: {e}")

    def _run(self, query: str, limit: int = 5, include_domains: list = None) -> list:
        if not self.exa_instance:
            return "Exa API not initialized due to missing API key."
        try:
            print(f"\n[ExaSearchTool] Searching Exa for query: '{query}'...")
            response = self.exa_instance.search(
                query=query,
                num_results=limit,
                include_domains=include_domains if include_domains else [],
            )
            results = []
            if response and response.results:
                for result in response.results:
                    results.append({"title": result.title, "url": result.url, "id": result.id})
                print(f"[ExaSearchTool] Found {len(results)} results for '{query}'.")
            else:
                print(f"[ExaSearchTool] No relevant results found for '{query}'.")
            return results
        except Exception as e:
            return f"An error occurred during Exa search: {e}"

class ExaContentRetrieveTool(BaseTool):
    name: str = "Exa Content Retrieval Tool"
    description: str = "Retrieves the full text content of a web page by its Exa result ID."
    exa_instance: Exa = None

    def __init__(self):
        super().__init__()
        try:
            self.exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
        except KeyError:
            raise ValueError("EXA_API_KEY environment variable not set. Please set it to your Exa API key in Colab Secrets.")
        except Exception as e:
            raise Exception(f"Failed to initialize ExaContentRetrieveTool: {e}")

    def _run(self, result_id: str) -> str:
        if not self.exa_instance:
            return "Exa API not initialized due to missing API key."
        try:
            print(f"[ExaContentRetrieveTool] Getting content for ID: {result_id}...")
            response = self.exa_instance.get_contents(ids=[result_id])
            if response and response.results:
                content = response.results[0].text
                print(f"[ExaContentRetrieveTool] Content retrieved for ID: {result_id}.")
                return content
            else:
                print(f"[ExaContentRetrieveTool] No content found for ID: {result_id}.")
                return "No content found for this ID."
        except Exception as e:
            return f"An error occurred while getting content: {e}"

try:
    exa_search_tool = ExaSearchTool()
    exa_content_tool = ExaContentRetrieveTool()
except ValueError as e:
    print(f"Tool initialization error: {e}")
    print("Please ensure your EXA_API_KEY environment variable is set in Colab Secrets and restart the script.")
    exit()

# --- NEW: 1b. Define Simulated Weather API Tool ---

class WeatherAPITool(BaseTool):
    """
    A simulated tool that provides a structured 7-day weather forecast.
    In a real application, this would integrate with a third-party weather API.
    """
    name: str = "Weather API Tool"
    description: str = "Provides a structured 7-day weather forecast for a specified location and starting date in 'Month Day, Year' format."

    def _run(self, location: str, start_date_str: str) -> str:
        # This is a simulated response. In a real scenario, you'd call a weather API.
        try:
            start_date = datetime.strptime(start_date_str, "%B %d, %Y")
        except ValueError:
            return "Error: start_date_str must be in 'Month Day, Year' format (e.g., 'June 3, 2025')."

        forecast_data = []

        # Define a plausible, static forecast for the example (consistent with previous web search results)
        forecast_template = [
            {"date_offset": 0, "conditions": "Sunny", "high": 24, "low": 14, "precip_chance": "10%"},
            {"date_offset": 1, "conditions": "Partly Cloudy", "high": 21, "low": 13, "precip_chance": "20%"},
            {"date_offset": 2, "conditions": "Light Rain", "high": 19, "low": 12, "precip_chance": "60%"},
            {"date_offset": 3, "conditions": "Cloudy", "high": 20, "low": 11, "precip_chance": "30%"},
            {"date_offset": 4, "conditions": "Mostly Sunny", "high": 23, "low": 14, "precip_chance": "15%"},
            {"date_offset": 5, "conditions": "Sunny", "high": 25, "low": 15, "precip_chance": "5%"},
            {"date_offset": 6, "conditions": "Partly Cloudy", "high": 22, "low": 13, "precip_chance": "25%"},
        ]

        for entry in forecast_template:
            forecast_date = start_date + timedelta(days=entry["date_offset"])
            forecast_data.append(
                f"* {forecast_date.strftime('%A, %B %d, %Y')}: "
                f"High: {entry['high']}°C, Low: {entry['low']}°C, "
                f"Conditions: {entry['conditions']}, Precipitation: {entry['precip_chance']}"
            )

        return (
            f"7-Day Weather Forecast for {location} starting {start_date_str}:\n"
            + "\n".join(forecast_data)
        )

# Instantiate the new Weather API tool
weather_api_tool = WeatherAPITool()


# --- 2. Define the Agents ---
# Explicitly setting llm=ChatOpenAI(model='gpt-3.5-turbo') for each agent

# Define the GPT-3.5-turbo LLM instance
# Set temperature to 0.7 for creativity, or 0.0 for more deterministic output
gpt_35_turbo_llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)

# Flight & Transportation Agent (FTA)
flight_agent = Agent(
    role='Flight Information Specialist',
    goal='Accurately identify relevant flight news, policies, and reliable booking links for given routes and dates.',
    backstory=(
        "You are an expert in air travel, meticulously researching current airline policies, "
        "flight search best practices, and airport information. Your precision ensures "
        "travelers receive the most accurate and helpful information for their flight planning."
    ),
    tools=[exa_search_tool, exa_content_tool],
    verbose=False, # Set to False to reduce output
    allow_delegation=False,
    llm=gpt_35_turbo_llm
)

# Itinerary Planning & Discovery Agent (IPDA)
itinerary_agent = Agent(
    role='Itinerary Planning & Discovery Agent',
    goal='Suggest engaging and relevant activities, attractions, and cultural experiences for a given destination and duration.',
    backstory=(
        "You are a creative travel planner, an expert in discovering unique and popular "
        "activities globally. You understand user preferences and can craft compelling "
        "daily itineraries, suggesting must-see sights and hidden gems."
    ),
    tools=[exa_search_tool, exa_content_tool],
    verbose=False, # Set to False to reduce output
    allow_delegation=True,
    llm=gpt_35_turbo_llm
)

# Real-time & Event Agent (RTEA) - UPDATED to use WeatherAPITool primarily for weather
realtime_agent = Agent(
    role='Real-time & Event Information Agent',
    goal='Provide up-to-date weather forecasts, local events, and any critical real-time travel alerts for a specified location and period.',
    backstory=(
        "You are a current events and weather oracle, constantly monitoring live data feeds "
        "and news for your specified location. Your job is to ensure travelers are "
        "fully prepared for local conditions and don't miss out on important happenings."
    ),
    # Now includes WeatherAPITool for structured weather, and ExaSearchTool for general events/alerts
    tools=[weather_api_tool, exa_search_tool],
    verbose=False, # Set to False to reduce output
    allow_delegation=False,
    llm=gpt_35_turbo_llm
)

# Travel Research Orchestrator Agent
orchestrator_agent = Agent(
    role='Travel Research Orchestrator',
    goal='Coordinate information gathering for comprehensive travel planning, delegating to specialized agents and synthesizing findings.',
    backstory=(
        "You are the central brain of a travel planning AI, capable of understanding complex user requests, "
        "breaking them down, and intelligently delegating tasks to expert agents. You then synthesize "
        "their findings into a coherent, actionable travel brief."
    ),
    tools=[],
    verbose=False, # Set to False to reduce output
    allow_delegation=True,
    llm=gpt_35_turbo_llm
)

# --- 3. Define the Tasks ---

# Task 1: Flight Research
flight_research_task = Task(
    description=(
        "Search for general flight information for a trip from **Montreal to Paris** "
        "during **July 2025**. Focus on finding reputable flight aggregators, "
        "news about typical flight prices, and any common airlines on this route. "
        "If a specific article seems highly relevant for price trends, retrieve its content."
    ),
    expected_output=(
        "A bullet-point summary of general flight information for Montreal to Paris in July 2025, "
        "including typical price ranges (if found), common airlines, and direct vs. connecting options. "
        "Include 2-3 relevant URLs from reputable flight search sites or travel news."
    ),
    agent=flight_agent,
    tools=[exa_search_tool, exa_content_tool]
)

# Task 2: Activity Planning for Paris
paris_activities_task = Task(
    description=(
        "Plan a week's worth of activities for a trip to **Paris**. "
        "Suggest a mix of popular attractions (e.g., Eiffel Tower, Louvre) and "
        "some cultural experiences or less touristy gems. "
        "Provide a daily breakdown of suggested activities for 7 days."
    ),
    expected_output=(
        """
        ## Paris One-Week Itinerary

        **Day 1: Iconic Paris**
        * Morning: [Activity 1]
        * Afternoon: [Activity 2]
        * Evening: [Activity 3]

        **Day 2: Art & Culture**
        * Morning: [Activity 1]
        * Afternoon: [Activity 2]
        * Evening: [Activity 3]

        ... (continue for all 7 days with specific suggestions) ...

        **Day 7: [Theme for Day 7]**
        * Morning: [Activity 1]
        * Afternoon: [Activity 2]
        * Evening: [Activity 3]

        List key attractions with brief descriptions and suggested visit times.
        """
    ),
    agent=itinerary_agent,
    tools=[exa_search_tool, exa_content_tool]
)

# NEW/UPDATED: Task 3: Weather Lookup for Paris
# The agent will now primarily use the WeatherAPITool for this.
current_date_for_task = datetime.now().strftime("%B %d, %Y") # Uses current date for the task

paris_weather_task = Task(
    description=(
        f"Get the 7-day weather forecast for **Paris, France** "
        f"starting **{current_date_for_task}**. "
        "Use the 'Weather API Tool' for structured weather data. "
        "Summarize the high/low temperatures, general conditions, and precipitation chances for each day."
    ),
    expected_output=(
        f"""
        ## Paris Weather Forecast (Next 7 Days - Starting {current_date_for_task})

        * {datetime.now().strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=1)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=2)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=3)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=4)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=5)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=6)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        """
    ),
    agent=realtime_agent,
    tools=[weather_api_tool] # <-- Explicitly use the new WeatherAPITool for this task
)

# Task 4: Orchestrator's Final Summary Task (Existing)
orchestrator_summary_task = Task(
    description=(
        "Review all research findings provided by the specialized agents (flight, activities, weather). "
        "Compile a comprehensive, actionable travel plan for the user for Paris in July 2025 (flights) "
        "and activities/weather for a week. "
        "Combine all information into a single, well-structured, user-friendly report."
    ),
    expected_output=(
        f"""
        ## Comprehensive Paris Travel Plan (July 2025 & Weekly Activities)

        ### Flight Information: Montreal to Paris - July 2025
        * **Route:** [Summary of main route, e.g., Montreal (YUL) to Paris (CDG)]
        * **Travel Period:** July 2025
        * **Likely Airlines:** [List common airlines found]
        * **Direct Flight Duration:** [Estimated duration]
        * **Estimated Price Range:** [Typical price range, e.g., $600 - $1,200 (Booking early recommended)]
        * **Recommended Booking Sites:** [List 2-3 reputable sites]

        ### Paris One-Week Itinerary
        * **Day 1: Iconic Paris**
            * Morning: [Activity 1]
            * Afternoon: [Activity 2]
            * Evening: [Activity 3]
        * **Day 2: Art & Culture**
            * Morning: [Activity 1]
            * Afternoon: [Activity 2]
            * Evening: [Activity 3]
        * ... (continue for all 7 days with specific suggestions) ...

        ### Paris Weather Forecast (Next Week: starting {current_date_for_task})
        * {datetime.now().strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=1)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=2)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=3)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=4)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=5)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]
        * {(datetime.now() + timedelta(days=6)).strftime("%A, %B %d, %Y")}: [High/Low temp, conditions, precipitation chance]

        ---
        *Action:* Start searching and comparing flights on the recommended platforms to secure your booking. Finalize your itinerary details based on your interests and book any tours or tickets in advance. Pack accordingly for the forecasted weather.
        """
    ),
    agent=orchestrator_agent,
)

# --- 4. Create and Run the Crew ---

travel_planning_crew = Crew(
    agents=[flight_agent, itinerary_agent, realtime_agent],
    tasks=[flight_research_task, paris_activities_task, paris_weather_task, orchestrator_summary_task],
    process=Process.hierarchical,
    manager_agent=orchestrator_agent,
    verbose=False # <-- Changed to False
)

print("\n\n############################################")
print("## Starting the Comprehensive TTPA Crew... ##")
print("############################################\n")

final_result = travel_planning_crew.kickoff()

print("\n############################################")
print("## TTPA Crew Finished! Final Output: ##")
print("############################################\n")
print(final_result)



############################################
## Starting the Comprehensive TTPA Crew... ##
############################################


[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flight July 2025 flight aggregator'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris July 2025 flight article'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flight July 2025'...
[ExaContentRetrieveTool] Getting content for ID: b9771ddd545104fd01bbe1d9a8b61dfa...
[ExaContentRetrieveTool] Getting content for ID: 123456789...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flight information July 2025'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025'...

############################################
## TTPA Crew Finished! Final Output: ##
############################################

## Comprehensive Paris Travel Plan (July 2025 & We